In [2]:
import sqlite3

import numpy as np
import pandas as pd
from faker import Faker


def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

## degrees 

In [3]:
conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)
df


,StudentID,Name,Degree,Exams,Scores
0,1,"Rodriguez, Pamela",graduate,"exam7 (2017), exam9 (2018), exam3 (2018), exam...","61, 38, 85, 70, 44, 43, 68"
1,2,"Jackson, Kristie",undergraduate,"exam2 (2017), exam2 (2017), exam8 (2018), exam...","66, 69, 85, 62, 92, 72, 44, 50, 23"
2,3,"Curtis, George",graduate,"exam7 (2017), exam6 (2017), exam2 (2017), exam...","68, 73, 62, 72"
3,4,"Casey, Christie",undergraduate,"exam10 (2019), exam4 (2019), exam4 (2019), exa...","41, 52, 52, 70"
4,5,"Yoder, Emily",undergraduate,"exam4 (2019), exam4 (2019), exam4 (2019)","52, 45, 61"
...,...,...,...,...,...
95,96,"Dyer, Benjamin",undergraduate,"exam10 (2019), exam4 (2019), exam6 (2017), exa...","47, 53, 64, 57, 71, 79, 77, 56"
96,97,"Delgado, Jason",undergraduate,"exam1 (2016), exam8 (2018)","72, 83"
97,98,"Cortez, Kenneth",undergraduate,"exam3 (2018), exam5 (2020), exam6 (2017)","83, 55, 71"
98,99,"Short, Elizabeth",undergraduate,"exam3 (2018), exam9 (2018), exam3 (2018), exam...","87, 34, 84, 75, 80, 60, 48, 55, 75"


## create_df_exams

In [32]:
conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)

exam_year = []
for index, row in df.iterrows():
    lst = row.values
    exams = [tuple(i.split(" ")) for i in lst[3].split(', ')]
    exam_year.extend([(exam, int(year[1:-1])) for exam, year in exams])

df2 = pd.DataFrame(exam_year, columns=['Exam', 'Year'])
df2 = df2.sort_values(by = ['Exam']).reset_index(drop = True)

df_exams = df2.drop_duplicates().reset_index(drop = True)


df_exams

,Exam,Year
0,exam1,2016
1,exam10,2019
2,exam2,2017
3,exam3,2018
4,exam4,2019
5,exam5,2020
6,exam6,2017
7,exam7,2017
8,exam8,2018
9,exam9,2018


In [5]:
conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)
exam_list = df["Exams"].unique()

exam_name = []
exam_date = []
exam_hasho = {}
# for i in range(len(exam_list)):
#     temp = exam_list[i].strip().split(",")
#     for i in range(len(temp)):
#         if temp[i].strip().split(" ")[0] not in exam_name:
#             if temp[i].strip().split(" ")[0] != " ":
#                 exam_name.append(temp[i].strip().split(" ")[0])
#         if temp[i].strip().split(" ")[1] not in exam_date:
#             if temp[i].strip().split(" ")[1] != " ":
#                 exam_date.append(temp[i].strip().split(" ")[1])
# exam_date

unique_exam_and_date = []
for i in range(len(exam_list)):
    exam_and_date = exam_list[i].split(",")
    for i in range(len(exam_and_date)):
        if exam_and_date[i].strip() not in unique_exam_and_date:
            unique_exam_and_date.append(exam_and_date[i].strip())

exam_hasho = {"Exam": [], "Year": []}

for exam in unique_exam_and_date:
    exam_name, year = exam.split(" (")
    exam_hasho["Exam"].append(exam_name)
    exam_hasho["Year"].append(year[:-1])     
df_exams = pd.DataFrame(exam_hasho)
df_exams
    
    
    

,Exam,Year
0,exam7,2017
1,exam9,2018
2,exam3,2018
3,exam5,2020
4,exam10,2019
5,exam1,2016
6,exam2,2017
7,exam8,2018
8,exam6,2017
9,exam4,2019


In [6]:
unique_exam_and_date.sort()

In [7]:
exam_hasho = {"Exam": [], "Year": []}

for exam in unique_exam_and_date:
    exam_name, year = exam.split(" (")
    exam_hasho["Exam"].append(exam_name)
    exam_hasho["Year"].append(year[:-1])

exam_hasho

{'Exam': ['exam1',
  'exam10',
  'exam2',
  'exam3',
  'exam4',
  'exam5',
  'exam6',
  'exam7',
  'exam8',
  'exam9'],
 'Year': ['2016',
  '2019',
  '2017',
  '2018',
  '2019',
  '2020',
  '2017',
  '2017',
  '2018',
  '2018']}

In [8]:
exams_hasho = { 
               "Exam" : [], 
               "Year" : []
               }

for i in range(len(unique_exam_and_date)):
    exam_hasho["Exam"].append(unique_exam_and_date[i].split(" ")[0])
    exam_hasho["Year"].append(unique_exam_and_date[i].split(" ")[1].replace("(", "").replace(")", ""))
exam_hasho   

{'Exam': ['exam1',
  'exam10',
  'exam2',
  'exam3',
  'exam4',
  'exam5',
  'exam6',
  'exam7',
  'exam8',
  'exam9',
  'exam1',
  'exam10',
  'exam2',
  'exam3',
  'exam4',
  'exam5',
  'exam6',
  'exam7',
  'exam8',
  'exam9'],
 'Year': ['2016',
  '2019',
  '2017',
  '2018',
  '2019',
  '2020',
  '2017',
  '2017',
  '2018',
  '2018',
  '2016',
  '2019',
  '2017',
  '2018',
  '2019',
  '2020',
  '2017',
  '2017',
  '2018',
  '2018']}

In [9]:
# for exam_year in exam_list[0].split(","):
#     for i in range(len(exam_year)):
#         print(exam_year.split(" "))
# temp = exam_list[34].strip().split(",")
# for i in range(len(temp)):
#     print(temp[i].strip().split(" ")[0])
exam_list[0].split(",")

['exam7 (2017)',
 ' exam9 (2018)',
 ' exam3 (2018)',
 ' exam7 (2017)',
 ' exam5 (2020)',
 ' exam10 (2019)',
 ' exam1 (2016)']

In [10]:

exam_list= []
unique_exam_and_date = []

conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)
exam_list = df["Exams"].unique()

exam_hasho = {"Exam": [], "Year": []}
for i in range(len(exam_list)):
    exam_and_date = exam_list[i].split(",")
    for i in range(len(exam_and_date)):
        if exam_and_date[i].strip() not in unique_exam_and_date:
            unique_exam_and_date.append(exam_and_date[i].strip())
unique_exam_and_date.sort()


for exam in unique_exam_and_date:
    exam_name, year = exam.split(" (")
    exam_hasho["Exam"].append(exam_name)
    exam_hasho["Year"].append(year[:-1])     

df_2 = pd.DataFrame.from_dict(exam_hasho)
df_2

,Exam,Year
0,exam1,2016
1,exam10,2019
2,exam2,2017
3,exam3,2018
4,exam4,2019
5,exam5,2020
6,exam6,2017
7,exam7,2017
8,exam8,2018
9,exam9,2018


In [11]:
exam_list= []
unique_exam_and_date = []

conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)
exam_list = df["Exams"].unique()

exam_hasho = {"Exam": [], "Year": []}
for i in range(len(exam_list)):
    exam_and_date = exam_list[i].split(",")
    for i in range(len(exam_and_date)):
        if exam_and_date[i].strip() not in unique_exam_and_date:
            unique_exam_and_date.append(exam_and_date[i].strip())
unique_exam_and_date.sort()


for exam in unique_exam_and_date:
    exam_name, year = exam.split(" (")
    exam_hasho["Exam"].append(exam_name)
    exam_hasho["Year"].append(int(year[:-1]))     
df_2 = pd.DataFrame.from_dict(exam_hasho)
df_2

,Exam,Year
0,exam1,2016
1,exam10,2019
2,exam2,2017
3,exam3,2018
4,exam4,2019
5,exam5,2020
6,exam6,2017
7,exam7,2017
8,exam8,2018
9,exam9,2018


In [12]:
df_exams_from_file = pd.read_csv("df_exams.csv")
df_exams_from_file

,Exam,Year
0,exam1,2016
1,exam10,2019
2,exam2,2017
3,exam3,2018
4,exam4,2019
5,exam5,2020
6,exam6,2017
7,exam7,2017
8,exam8,2018
9,exam9,2018


## create_df_students

In [13]:
conn = create_connection("non_normalized.db")
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)


student_names = df["Name"]
student_degrees = df["Degree"]


students_hasho = {"StudentID": [], "First_Name": [], "Last_Name" : [], "Degree": []}


for i in range(len(student_names)):
    students_hasho["StudentID"].append(i+1)
    students_hasho["First_Name"].append(student_names[i].split(", ")[1])
    students_hasho["Last_Name"].append(student_names[i].split(", ")[0])
    students_hasho["Degree"].append(student_degrees[i])
    
df_students = pd.DataFrame.from_dict(students_hasho)
df_students

,StudentID,First_Name,Last_Name,Degree
0,1,Pamela,Rodriguez,graduate
1,2,Kristie,Jackson,undergraduate
2,3,George,Curtis,graduate
3,4,Christie,Casey,undergraduate
4,5,Emily,Yoder,undergraduate
...,...,...,...,...
95,96,Benjamin,Dyer,undergraduate
96,97,Jason,Delgado,undergraduate
97,98,Kenneth,Cortez,undergraduate
98,99,Elizabeth,Short,undergraduate


In [14]:

student_degrees

0          graduate
1     undergraduate
2          graduate
3     undergraduate
4     undergraduate
          ...      
95    undergraduate
96    undergraduate
97    undergraduate
98    undergraduate
99    undergraduate
Name: Degree, Length: 100, dtype: object

## create_df_studentexamscores

In [15]:
conn = create_connection("non_normalized.db")
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)

student_exams = df["Exams"]
student_scores = df["Scores"]
student_ids = df["StudentID"]

exam_scores_hasho = {
    "StudentID" : [],
    "Exam" : [],
    "Score" : []
}

for i in range(len(student_exams)):
    ith_student_exams = student_exams[i].split(" ")[::2]
    ith_student_scores = student_scores[i].split(", ")
    for j in range(len(ith_student_exams)):
        exam_scores_hasho["StudentID"].append(i+1)
        exam_scores_hasho['Exam'].append(ith_student_exams[j])
        exam_scores_hasho["Score"].append(int(ith_student_scores[j]))

df_studentexamscores = pd.DataFrame.from_dict(exam_scores_hasho)
df_studentexamscores
        

,StudentID,Exam,Score
0,1,exam7,61
1,1,exam9,38
2,1,exam3,85
3,1,exam7,70
4,1,exam5,44
...,...,...,...
579,99,exam4,60
580,99,exam10,48
581,99,exam4,55
582,99,exam7,75


In [16]:
ith_student_exams = student_exams[0].split(" ")[::2]

In [17]:
ith_student_scores =( student_scores[0].split(", "))
ith_student_scores

['61', '38', '85', '70', '44', '43', '68']

## ex 1

In [18]:
df_exams

,Exam,Year
0,exam7,2017
1,exam9,2018
2,exam3,2018
3,exam5,2020
4,exam10,2019
5,exam1,2016
6,exam2,2017
7,exam8,2018
8,exam6,2017
9,exam4,2019


In [19]:
df_exams.sort_values(by='Year')


,Exam,Year
5,exam1,2016
0,exam7,2017
6,exam2,2017
8,exam6,2017
1,exam9,2018
2,exam3,2018
7,exam8,2018
4,exam10,2019
9,exam4,2019
3,exam5,2020


## ex2

In [20]:
df_students

,StudentID,First_Name,Last_Name,Degree
0,1,Pamela,Rodriguez,graduate
1,2,Kristie,Jackson,undergraduate
2,3,George,Curtis,graduate
3,4,Christie,Casey,undergraduate
4,5,Emily,Yoder,undergraduate
...,...,...,...,...
95,96,Benjamin,Dyer,undergraduate
96,97,Jason,Delgado,undergraduate
97,98,Kenneth,Cortez,undergraduate
98,99,Elizabeth,Short,undergraduate


In [21]:
df_degree_count = df_students['Degree'].value_counts().reset_index()
df_degree_count = df_degree_count.rename(columns={'index':'Count'})
df_degree_count

,Count,Degree
0,undergraduate,65
1,graduate,35


In [22]:
# df.columns = pd.MultiIndex.from_frame(df)
# df = pd.MultiIndex.from_frame(df)
df
df.set_index(0,inplace=True)

df = df_students["Degree"].value_counts().to_frame()
df.columns=[['Count']]



KeyError: 'None of [0] are in the columns'

In [ ]:
df_file = pd.read_csv("ex2.csv", index_col=0)
df_file.columns = [['Count']]
df_file 

,Count
undergraduate,65
graduate,35


In [ ]:
from pandas.testing import assert_frame_equal

assert_frame_equal(df, df_file)

AssertionError: DataFrame Expected type <class 'pandas.core.frame.DataFrame'>, found <class 'pandas.core.indexes.multi.MultiIndex'> instead

In [ ]:
df = pd.MultiIndex(df)


TypeError: Must pass both levels and codes

In [ ]:
df_studentexamscores

,StudentID,Exam,Score
0,1,exam7,61
1,1,exam9,38
2,1,exam3,85
3,1,exam7,70
4,1,exam5,44
...,...,...,...
579,99,exam4,60
580,99,exam10,48
581,99,exam4,55
582,99,exam7,75


## ex 3

In [ ]:
# Merge df_studentexamscores and df_exams
df = pd.merge(df_studentexamscores, df_exams, on='Exam')

# Group by student and exam, and compute the mean score
df_averages = df.groupby(['Exam', 'Year'])['Score'].mean().round(2).reset_index()
# Sort the dataframe in descending order by the average score
df_averages.sort_values(by='Score', ascending=False, inplace=True)
df_averages["Score"].astype("int32")
df_averages["Year"] = df_averages["Year"].astype("int32")

df_averages = df_averages.rename(columns={'Score': 'average'})
df_averages.set_index('Exam', inplace=True)
df_averages

,Year,average
Exam,,
exam3,2018,86.16
exam8,2018,80.18
exam1,2016,75.09
exam7,2017,68.96
exam2,2017,64.28
exam6,2017,61.80
exam4,2019,54.04
exam5,2020,45.56
exam10,2019,44.68


In [ ]:
df_from_file = pd.read_csv("ex3.csv", index_col=0)
df_from_file['Year'] = df_from_file['Year'].astype('int32')
df_from_file

,Year,average
Exam,,
exam3,2018,86.16
exam8,2018,80.18
exam1,2016,75.09
exam7,2017,68.96
exam2,2017,64.28
exam6,2017,61.80
exam4,2019,54.04
exam5,2020,45.56
exam10,2019,44.68


In [ ]:
from pandas.testing import assert_frame_equal

assert_frame_equal(df_averages, df_from_file)

## Ex 4

In [ ]:
df_studentexamscores

,StudentID,Exam,Score
0,1,exam7,61
1,1,exam9,38
2,1,exam3,85
3,1,exam7,70
4,1,exam5,44
...,...,...,...
579,99,exam4,60
580,99,exam10,48
581,99,exam4,55
582,99,exam7,75


In [ ]:
df_studentexamscores

,StudentID,Exam,Score
0,1,exam7,61
1,1,exam9,38
2,1,exam3,85
3,1,exam7,70
4,1,exam5,44
...,...,...,...
579,99,exam4,60
580,99,exam10,48
581,99,exam4,55
582,99,exam7,75


In [ ]:
df_students

,StudentID,First_Name,Last_Name,Degree
0,1,Pamela,Rodriguez,graduate
1,2,Kristie,Jackson,undergraduate
2,3,George,Curtis,graduate
3,4,Christie,Casey,undergraduate
4,5,Emily,Yoder,undergraduate
...,...,...,...,...
95,96,Benjamin,Dyer,undergraduate
96,97,Jason,Delgado,undergraduate
97,98,Kenneth,Cortez,undergraduate
98,99,Elizabeth,Short,undergraduate


In [ ]:
df = pd.merge(df_studentexamscores, df_students, on='StudentID')
df


# Group by student and exam, and compute the mean degree
df_averages = df.groupby(['StudentID', 'Exam'])['Degree'].mean().reset_index()
df_averages


TypeError: Could not convert graduate to numeric

In [ ]:
df_studentexamscores

,StudentID,Exam,Score
0,1,exam7,61
1,1,exam9,38
2,1,exam3,85
3,1,exam7,70
4,1,exam5,44
...,...,...,...
579,99,exam4,60
580,99,exam10,48
581,99,exam4,55
582,99,exam7,75


In [ ]:
df = pd.merge(df_studentexamscores, df_exams, on='Exam')

# Group by student and exam, and compute the mean score
df_averages = df.groupby(['Exam', 'Year'])['Score'].mean().round(2).reset_index()
# Sort the dataframe in descending order by the average score
df_averages.sort_values(by='Score', ascending=False, inplace=True)
df_averages["Score"].astype("int32")
df_averages["Year"] = df_averages["Year"].astype("int32")

df_averages = df_averages.rename(columns={'Score': 'average'})
df_averages.set_index('Exam', inplace=True)
df_averages

,Year,average
Exam,,
exam3,2018,86.16
exam8,2018,80.18
exam1,2016,75.09
exam7,2017,68.96
exam2,2017,64.28
exam6,2017,61.80
exam4,2019,54.04
exam5,2020,45.56
exam10,2019,44.68


In [39]:
df = (
    pd.merge(df_studentexamscores, df_students, on = 'StudentID')
    .groupby(['Degree'])
    .mean()
    .round(2)
    .rename(columns = {'Score': 'Average'})
    )

df['Average'].to_frame()


,Average
Degree,
graduate,62.50
undergraduate,61.63


In [52]:
temp_df = (
    pd.merge(df_studentexamscores, df_students, on = 'StudentID')
    .groupby(['StudentID'])
    .mean()
    .round(2)
    .reset_index()
    .sort_values(by = 'Score', ascending = False)
    .rename(columns = {'Score': 'average'})
    )

df = pd.merge(temp_df, df_students, on = 'StudentID')
df[['First_Name', 'Last_Name', 'Degree','average']].head(10)
df


,StudentID,average,First_Name,Last_Name,Degree
0,95,83.00,John,Washington,graduate
1,100,82.00,Robert,Andrade,undergraduate
2,18,79.00,Paul,Smith,undergraduate
3,97,77.50,Jason,Delgado,undergraduate
4,92,75.00,Calvin,Perez,undergraduate
...,...,...,...,...,...
95,85,47.33,William,Fowler,undergraduate
96,93,47.00,Helen,Davis,undergraduate
97,36,46.25,Samantha,Erickson,undergraduate
98,49,45.33,Eric,Schaefer,undergraduate


In [61]:
df = pd.merge(df_studentexamscores, df_exams, on='Exam')

df_averages = (
    df
    .groupby(['Exam', 'Year'])[
    'Score']
    .mean()
    .round(2)
    .reset_index()
    )

df_averages.sort_values(by='Score', ascending=False, inplace=True)

df_averages["Year"] = df_averages["Year"].astype("int32")

df_averages = df_averages.rename(columns={'Score': 'average'})
df_averages.set_index('Exam', inplace=True)

df_averages

,Year,average
Exam,,
exam3,2018,86.16
exam8,2018,80.18
exam1,2016,75.09
exam7,2017,68.96
exam2,2017,64.28
exam6,2017,61.80
exam4,2019,54.04
exam5,2020,45.56
exam10,2019,44.68
